<a href="https://colab.research.google.com/github/Ilektra-Giassa/Tutorials/blob/main/realEstate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#How much more (inflation)?
A regression problem

In [26]:
from keras.datasets import boston_housing
import numpy as np
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
train_data.shape #(404, 13) various real estate features
test_data.shape #(102, 13)
train_targets.shape #(404,) prices
test_targets.shape #(102,)

(102,)

### Normalize datasets

In [31]:
np.max(train_targets)
np.min(train_targets)
#prices between 5K and 50K :S

5.0

In [34]:
#calculate the avg and std per row of the train data -> elem_new=(elem-avg)/std
mean=train_data.mean(axis=0)
train_data-=mean
std=train_data.std(axis=0)
train_data/=std

### Function to build the net

In [53]:
from keras import layers, models
def build_model():
  model=models.Sequential()
  model.add(layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  #compile the net
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

### Small train dataset, sooo K-fold validation

In [59]:
#let's do 4-fold
k=4
num_val_samples=len(train_data)//4 #101 validation samples
num_epochs=100
all_scores=[]
for i in range(k):
  #prepare validation data
  print('Processing fold #', i)
  val_data=train_data[i*num_val_samples: (i+1)*num_val_samples]
  val_targets=train_targets[i*num_val_samples: (i+1)*num_val_samples]

  #prepare training data
  partial_train_data=np.concatenate(
      [train_data[:i*num_val_samples],
       train_data[(i+1)*num_val_samples:]], axis=0)
  partial_train_targets=np.concatenate(
      [train_targets[:i*num_val_samples],
       train_targets[(i+1)*num_val_samples:]], axis=0)
  
  #build model per fold
  model=build_model()
  model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae=model.evaluate(val_data, val_targets, verbose=0) #verbose=0 -> silent mode
  all_scores.append(val_mae) #mean absolute error

Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3


In [62]:
np.mean(all_scores)
#for num_epochs=100, mean(mae)~2.4K which comparable to the 5K-50K price of real estate
#if we try more epochs, e.g. 500, mean(mae)~

2.8600950241088867

Increase the epochs for "optimal" mae, let's do it again and keep track of history

In [ ]:
num_epochs=500
all_mae_histories=[]
for in range(k):
  print('Processing fold #', i)
  